# Vision Agents with smolagents


This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

Empowering agents with visual capabilities is crucial for solving tasks that go beyond text processing. Many real-world challenges, such as web browsing or document understanding, require analyzing rich visual content. Fortunately, smolagents provides built-in support for vision-language models (VLMs), enabling agents to process and interpret images effectively.

In this example, imagine Alfred, the butler at Wayne Manor, is tasked with verifying the identities of the guests attending the party. As you can imagine, Alfred may not be familiar with everyone arriving. To help him, we can use an agent that verifies their identity by searching for visual information about their appearance using a VLM. This will allow Alfred to make informed decisions about who can enter.

## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [ ]:
!pip install smolagents

Let's also login to the Hugging Face Hub to have access to the Inference API.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Providing Images at the Start of the Agent's Execution

In this approach, images are passed to the agent at the start and stored as `task_images` alongside the task prompt. The agent then processes these images throughout its execution.  

Consider the case where Alfred wants to verify the identities of the superheroes attending the party. He already has a dataset of images from previous parties with the names of the guests. Given a new visitor's image, the agent can compare it with the existing dataset and make a decision about letting them in.  

In this case, a guest is trying to enter, and Alfred suspects that this visitor might be The Joker impersonating Wonder Woman. Alfred needs to verify their identity to prevent anyone unwanted from entering.  

Let’s build the example. First, the images are loaded. In this case, we use images from Wikipedia to keep the example minimal, but imagine the possible use-cases!

In [1]:
from PIL import Image
import requests
from io import BytesIO

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg",
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg"
]

images = []
for url in image_urls:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36" 
    }
    response = requests.get(url,headers=headers)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    images.append(image)

In [ ]:
for i, img in enumerate(images):
    img.save(f'immagine_{i+1}.jpg')
    print(f'Salvata immagine_{i+1}.jpg')

Now that we have the images, the agent will tell us wether the guests is actually a superhero (Wonder Woman) or a villian (The Joker).

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
from smolagents import CodeAgent, OpenAIServerModel

model = OpenAIServerModel(model_id="gpt-4o")

# Instantiate the agent
agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=20,
    verbosity_level=2
)

response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=images
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing and return the description. │
│     Tell me if the guest is The Joker or Wonder Woman.                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
I can't identify people, but I can describe the costume and makeup of the comic character in the images.           
                                                                                                                   
1. **Costume:**                                                                                                    
   - The character is wearing a purple suit with a large purple bow tie.                                           
   - There is a hint of an orange shirt underneath the suit.                                                       
                                                                                                                   
2. **Makeup:**                                                                                                     
   - The face is painted white.                                                                                    
   - Blue eyeshadow is applied heavily.                                                                            
   - The lips are painted bright red, and there's an exaggerated smile.                                            
                                                                                                                   
This description matches the character known as **The Joker**.                                                     

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            I can't identify people, but I can describe the costume and makeup of the comic character in the 
images.

1. **Costume:**
   - The character is wearing a purple suit with a large purple bow tie.
   - There is a hint of an orange shirt underneath the suit.

2. **Makeup:**
   - The face is painted white.
   - Blue eyeshadow is applied heavily.
   - The lips are painted bright red, and there's an exaggerated smile.

This description matches the character known as **The Joker**.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 6.73 seconds| Input tokens: 2,985 | Output tokens: 109]

...

[Step 14: Duration 70.67 seconds| Input tokens: 176,670 | Output tokens: 2,380]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 15 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-AVHG9pyQ1ICEKjIQ0jSdIhZW 
on tokens per min (TPM): Limit 30000, Requested 31112. The input or output tokens must be reduced in order to run 
successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': 
None, 'code': 'rate_limit_exceeded'}}

[Step 15: Duration 22.19 seconds]

AgentGenerationError: Error in generating model output:
Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-AVHG9pyQ1ICEKjIQ0jSdIhZW on tokens per min (TPM): Limit 30000, Requested 31112. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
response

{'description': '\n1. Costume:\n   - A purple suit with a yellow shirt and a large purple bow tie.\n   - Features a white flower lapel and a playing card in the second image.\n   - The style is flamboyant, consistent with a comic villain.\n\n2. Makeup:\n   - White face makeup covering the entire face.\n   - Red lips forming a wide, exaggerated smile.\n   - Blue eyeshadow with dark eye accents.\n   - Slicked-back green hair.\n',
 'character': 'The Joker'}

In this case, the output reveals that the person is impersonating someone else, so we can prevent The Joker from entering the party!

## Providing Images with Dynamic Retrieval

This examples is provided as a `.py` file since it needs to be run locally since it'll browse the web. Go to the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course) for more details.